In this notebook we will generate and save the data frames that we will use in visualization, using the clean ones done in the previous steps

In [40]:
import pandas as pd
sys.path.append('../')
import src.limpieza as lmp
from geopy.geocoders import Nominatim
import re
import geopandas as gpd
from keplergl import KeplerGl

In [2]:
df_civitatis_clean = pd.read_pickle(r'C:\Users\mituc\Ironhack\Curso\IronLabs\Proyecto-1\data\civitatis_clean.pkl')
df_turistas_admon = pd.read_pickle(r'C:\Users\mituc\Ironhack\Curso\IronLabs\Proyecto-1\data\Barcelona_cleaned_tourist.pkl')


In [41]:
df_civitatis_location = pd.read_csv(r"C:\Users\mituc\Ironhack\Curso\IronLabs\Proyecto-1\data\civitatis_location.csv", sep=',', encoding='latin-1')

In [8]:
df_civitatis_location.tail(5)

,procedencia,fecha,nombre,opinion,viajo_con,georeferenced_column,latitude,longitude,bar_lat,bar_lon
16226,"Porto, Portugal",2018-09-17 00:00,Antonio Augusto Mendes Moreira,Um tour pela cidade antiga muito interessante ...,ViajÃ³ en pareja,"(41.1494512, -8.6107884)",41.149451,-8.610788,41.382894,2.177432
16227,"New York, Estados Unidos",2018-09-02 00:00,Sergio Camargo,Pedro estuvo fantastico. Fue claro y ameno al ...,ViajÃ³ con hijos mayores,"(40.7127281, -74.0060152)",40.712728,-74.006015,41.382894,2.177432
16228,"Faro, Portugal",2018-09-01 00:00,Mariana Goncalves,NaN,ViajÃ³ con amigos,"(37.0162727, -7.9351771)",37.016273,-7.935177,41.382894,2.177432
16229,"Braga, Portugal",2018-06-28 00:00,JosÃ© Carlos Miranda Reis,O tour foi formidavel. A guia Laura e eficente...,ViajÃ³ en pareja,"(41.5510583, -8.4280045)",41.551058,-8.428005,41.382894,2.177432
16230,"Barcelona, EspaÃ±a",2017-09-10 00:00,Susanna,Som de Barcelona i hem descovert curiositats i...,ViajÃ³ en pareja,"(41.3828939, 2.1774322)",41.382894,2.177432,41.382894,2.177432


# Civitatis

Total number of tourist grouped by month

In [ ]:
lmp.total_civitatis_tourists_month(df_civitatis_clean)

In [15]:
df_civitatis_clean.shape

(16231, 5)

In [3]:
df_civitatis_clean.head(5)

,fecha,nombre,procedencia,opinion,viajo_con
0,2022-03-11,Ana,"Madrid, España","Victor es encantador, te explica todo al detal...",Viajó con hijos mayores
1,2022-03-11,Montserrat,España,"Casi 3 horas de tour pero Víctor, el guía que ...",Viajó solo
2,2022-03-11,Ainhoa,España,,Viajó en pareja
3,2022-03-11,Patricia Suárez Carrillo,"Madrid, España",A mi pareja y a mi nos ha encantado! Nuestro g...,Viajó en pareja
4,2022-03-11,Anónimo,,,


Longitude and latitude of the place of origin of tourists

In [4]:
def extraer_coordenadas(x):
    try:
        geolocator = Nominatim(user_agent = "pepe")
        location = geolocator.geocode(x)

        return location.latitude, location.longitude
    except:
        return 'no data'

In [5]:
df_coordinates = pd.DataFrame(df_civitatis_clean['procedencia'].unique())

In [ ]:
df_coordinates["georeferenced_column"] = df_coordinates[0].apply(extraer_coordenadas)

In [10]:
df_coordinates.to_pickle(r'C:\Users\mituc\Ironhack\Curso\IronLabs\Proyecto-1\data\lat_lon.pkl')


In [37]:
df_coordinates = pd.read_pickle(r'C:\Users\mituc\Ironhack\Curso\IronLabs\Proyecto-1\data\lat_lon.pkl')

In [12]:
df_coordinates.head(5)

,0,georeferenced_column,latitude,longitude
0,"Madrid, España","(40.4167047, -3.7035825)",40.416705,-3.703582
1,España,"(39.3260685, -4.8379791)",39.326068,-4.837979
2,,no data,NaN,NaN
3,"Alicante, España","(38.3436365, -0.4881708)",38.343637,-0.488171
4,"Quito, Ecuador","(-0.2201641, -78.5123274)",-0.220164,-78.512327


In [24]:
def lat_lon(df, column):
    df['latitude'] = range(len(df))
    df['longitude'] = range(len(df))
    for i in range(len(df)):
        df['latitude'].iloc[i] = column.iloc[i][0]
        df['longitude'].iloc[i] = column.iloc[i][1]
    df['latitude'] = pd.to_numeric(df['latitude'],errors='coerce')
    df['longitude'] = pd.to_numeric(df['longitude'],errors='coerce')
    return df


In [17]:
df_civitatis_clean.head(5)

,fecha,nombre,procedencia,opinion,viajo_con
0,2022-03-11,Ana,"Madrid, España","Victor es encantador, te explica todo al detal...",Viajó con hijos mayores
1,2022-03-11,Montserrat,España,"Casi 3 horas de tour pero Víctor, el guía que ...",Viajó solo
2,2022-03-11,Ainhoa,España,,Viajó en pareja
3,2022-03-11,Patricia Suárez Carrillo,"Madrid, España",A mi pareja y a mi nos ha encantado! Nuestro g...,Viajó en pareja
4,2022-03-11,Anónimo,,,


In [13]:
df_civitatis_clean.set_index(['procedencia'], inplace=True)
df_coordinates.set_index([0], inplace=True)

In [14]:
df_civitatis_location = df_civitatis_clean.join(df_coordinates, on='procedencia')

In [21]:
geolocator = Nominatim(user_agent="miguel")
location = geolocator.geocode("Barcelona, España")
df_civitatis_location['bar_lat'] = location.latitude
df_civitatis_location['bar_lon'] = location.longitude

In [16]:
df_civitatis_location['fecha'] = df_civitatis_location['fecha'].astype(str) + ' 00:00'

In [17]:
df_civitatis_location['fecha'] = pd.to_datetime(df_civitatis_location['fecha'], format='%Y-%m-%d %H:%M') 

In [18]:
df_civitatis_location['fecha']=df_civitatis_location['fecha'].dt.strftime('%Y-%m-%d %H:%M')

In [267]:
df_civitatis_location.dtypes

procedencia              object
fecha                    object
nombre                   object
opinion                  object
viajo_con                object
georeferenced_column     object
latitude                float64
longitude               float64
bar_lat                 float64
bar_lon                 float64
dtype: object

In [23]:
df_civitatis_location.to_csv(r'C:\Users\mituc\Ironhack\Curso\IronLabs\Proyecto-1\data\civitatis_location.csv')

In [4]:
df_civitatis_location_primary = pd.read_csv(r'C:\Users\mituc\Ironhack\Curso\IronLabs\Proyecto-1\data\civitatis_location_primary.csv')

In [18]:
df_civitatis_location_primary.drop(['Unnamed: 0'], axis=1, inplace=True)

In [11]:
df_num_loc =  df_civitatis_location_primary.groupby(['procedencia']).count()

In [12]:
df_num_loc.drop(['Unnamed: 0', 'fecha','georeferenced_column', 'latitude', 'longitude', 'periodo', 'opinion', 'viajo_con', 'bar_lat', 'bar_lon', 'primary_key'], axis=1, inplace=True)

In [16]:
df_num_loc.to_csv(r'C:\Users\mituc\Ironhack\Curso\IronLabs\Proyecto-1\data\civ_num_loc.csv')

In [39]:
df_num_loc.dtypes

procedencia    object
nombre          int64
dtype: object

In [38]:
df_num_loc['procedencia'].str.explode(',')

AttributeError: 'StringMethods' object has no attribute 'explode'

In [21]:
df_num_loc.reset_index(inplace=True)

In [67]:
df_oi = df_civitatis_location_primary.groupby(['periodo', 'georeferenced_column', 'latitude', 'longitude']).count()

In [68]:
df_oi.drop(['nombre','opinion', 'viajo_con', 'procedencia', 'bar_lat', 'bar_lon', 'primary_key'], axis=1, inplace=True)

In [73]:
df_oi['periodo'] = pd.to_datetime(df_oi['periodo'], format='%Y-%m-%d %H:%M') 

In [74]:
df_oi['periodo']=df_oi['periodo'].dt.strftime('%Y-%m-%d %H:%M')

In [71]:
df_oi.head(5)

,periodo,georeferenced_column,latitude,longitude,fecha
0,2017-07,"(-12.0621065, -77.0365256)",-12.062107,-77.036526,1
1,2017-07,"(-33.4377756, -70.6504502)",-33.437776,-70.650450,1
2,2017-07,"(-34.6075682, -58.4370894)",-34.607568,-58.437089,2
3,2017-07,"(21.1617854, -86.8510468)",21.161785,-86.851047,1
4,2017-07,"(25.7741728, -80.19362)",25.774173,-80.193620,1


In [70]:
df_oi.reset_index(inplace=True)

In [66]:
df_oi.to_csv(r'C:\Users\mituc\Ironhack\Curso\IronLabs\Proyecto-1\data\oi.csv')

In [2]:
df_oi = pd.read_csv(r'C:\Users\mituc\Ironhack\Curso\IronLabs\Proyecto-1\data\oi.csv')

In [42]:
map_1 = KeplerGl(height = 600, weight = 500, data = {"viajes": df_civitatis_location})

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [45]:
map_1.save_to_html(file_name='civitatis.html')

Map saved to civitatis.html!


In [44]:
map_1

KeplerGl(data={'viajes':                     procedencia             fecha  \
0               Madrid, EspaÃ±a …

In [26]:
df_civitatis_location.head(5)

,procedencia,fecha,nombre,opinion,viajo_con,georeferenced_column,latitude,longitude,bar_lat,bar_lon
0,"Madrid, España",2022-03-11 00:00,Ana,"Victor es encantador, te explica todo al detal...",Viajó con hijos mayores,"(40.4167047, -3.7035825)",40.416705,-3.703582,41.382894,2.177432
1,España,2022-03-11 00:00,Montserrat,"Casi 3 horas de tour pero Víctor, el guía que ...",Viajó solo,"(39.3260685, -4.8379791)",39.326068,-4.837979,41.382894,2.177432
2,España,2022-03-11 00:00,Ainhoa,NaN,Viajó en pareja,"(39.3260685, -4.8379791)",39.326068,-4.837979,41.382894,2.177432
3,"Madrid, España",2022-03-11 00:00,Patricia Suárez Carrillo,A mi pareja y a mi nos ha encantado! Nuestro g...,Viajó en pareja,"(40.4167047, -3.7035825)",40.416705,-3.703582,41.382894,2.177432
4,NaN,2022-03-11 00:00,Anónimo,NaN,NaN,no data,NaN,NaN,41.382894,2.177432


In [4]:
df_civitatis_location.drop(['Unnamed: 0'], axis=1, inplace=True)

# Spanish goverment

As civitatis started its activity for the tour that we are studying in June 2017, we'll select that period from the data frame we had generated using the data given by the goverment.


In [4]:
lmp.admon_cividates(df_turistas_admon)

In [5]:
df_turistas_admon_17 = pd.read_pickle(r'C:\Users\mituc\Ironhack\Curso\IronLabs\Proyecto-1\data\admon_cividates.pkl')

In [6]:
df_turistas_admon_17.head(5)

,periodo,viajeros_espana,viajeros_extranjero,pernoctaciones_espana,pernoctaciones_extranjero,media_pernoc_esp,media_pernoc_ext,total_viajeros,total_pernoctaciones,media_total_pernoc
55,2017-07,137560,634306,261185,1803145,1.9,2.8,771866,2064330,2.7
54,2017-08,92632,601050,194356,1807172,2.1,3.0,693682,2001528,2.9
53,2017-09,116521,568131,216184,1550853,1.9,2.7,684652,1767037,2.6
52,2017-10,125400,531660,238121,1434376,1.9,2.7,657060,1672497,2.5
51,2017-11,140384,391985,244155,1022604,1.7,2.6,532369,1266759,2.4
